In [1]:
from __future__ import print_function
import findspark
findspark.init()
findspark.find()
import pyspark
findspark.find()
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StandardScaler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import StringIndexer
from pyspark.ml.classification import GBTClassifier

ModuleNotFoundError: No module named 'findspark'

In [2]:
if __name__ == "__main__":
    spark = SparkSession\
        .builder\
        .appName("GradientBosstingClassifier")\
        .getOrCreate()

In [18]:
dataset = spark.read.csv("diabetes.csv",header=True)

In [19]:
dataset.show()

+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|          6|    148|           72|           35|      0|33.6|                   0.627| 50|      1|
|          1|     85|           66|           29|      0|26.6|                   0.351| 31|      0|
|          8|    183|           64|            0|      0|23.3|                   0.672| 32|      1|
|          1|     89|           66|           23|     94|28.1|                   0.167| 21|      0|
|          0|    137|           40|           35|    168|43.1|                   2.288| 33|      1|
|          5|    116|           74|            0|      0|25.6|                   0.201| 30|      0|
|          3|     78|           50|           32|     88|  31|                   0.248| 26|      1|


In [20]:
dataset.printSchema()

root
 |-- Pregnancies: string (nullable = true)
 |-- Glucose: string (nullable = true)
 |-- BloodPressure: string (nullable = true)
 |-- SkinThickness: string (nullable = true)
 |-- Insulin: string (nullable = true)
 |-- BMI: string (nullable = true)
 |-- DiabetesPedigreeFunction: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Outcome: string (nullable = true)



In [21]:
from pyspark.sql.functions import col
new_data = dataset.select(*(col(c).cast("float").alias(c) for c in dataset.columns))

In [22]:
new_data.printSchema()

root
 |-- Pregnancies: float (nullable = true)
 |-- Glucose: float (nullable = true)
 |-- BloodPressure: float (nullable = true)
 |-- SkinThickness: float (nullable = true)
 |-- Insulin: float (nullable = true)
 |-- BMI: float (nullable = true)
 |-- DiabetesPedigreeFunction: float (nullable = true)
 |-- Age: float (nullable = true)
 |-- Outcome: float (nullable = true)



In [23]:
from pyspark.sql.functions import col, count, isnan, when
#checking for null ir nan type values in our columns
new_data.select([count(when(col(c).isNull(), c)).alias(c) for c in new_data.columns]).show()

+-----------+-------+-------------+-------------+-------+---+------------------------+---+-------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin|BMI|DiabetesPedigreeFunction|Age|Outcome|
+-----------+-------+-------------+-------------+-------+---+------------------------+---+-------+
|          0|      0|            0|            0|      0|  0|                       0|  0|      0|
+-----------+-------+-------------+-------------+-------+---+------------------------+---+-------+



In [24]:
cols=new_data.columns
cols.remove("Outcome")
assembler = VectorAssembler(inputCols=cols,outputCol="features")
# Now let us use the transform method to transform our dataset
data=assembler.transform(new_data)
data = data.select("features",'Outcome')

In [25]:
data.show()

+--------------------+-------+
|            features|Outcome|
+--------------------+-------+
|[6.0,148.0,72.0,3...|    1.0|
|[1.0,85.0,66.0,29...|    0.0|
|[8.0,183.0,64.0,0...|    1.0|
|[1.0,89.0,66.0,23...|    0.0|
|[0.0,137.0,40.0,3...|    1.0|
|[5.0,116.0,74.0,0...|    0.0|
|[3.0,78.0,50.0,32...|    1.0|
|[10.0,115.0,0.0,0...|    0.0|
|[2.0,197.0,70.0,4...|    1.0|
|[8.0,125.0,96.0,0...|    1.0|
|[4.0,110.0,92.0,0...|    0.0|
|[10.0,168.0,74.0,...|    1.0|
|[10.0,139.0,80.0,...|    0.0|
|[1.0,189.0,60.0,2...|    1.0|
|[5.0,166.0,72.0,1...|    1.0|
|[7.0,100.0,0.0,0....|    1.0|
|[0.0,118.0,84.0,4...|    1.0|
|[7.0,107.0,74.0,0...|    1.0|
|[1.0,103.0,30.0,3...|    0.0|
|[1.0,115.0,70.0,3...|    1.0|
+--------------------+-------+
only showing top 20 rows



In [28]:
(train, test) = data.randomSplit([0.7, 0.3])

In [31]:
gradient_boost_class = GBTClassifier(labelCol="Outcome", featuresCol="features")

In [32]:
model = gradient_boost_class.fit(train)

In [33]:
predictions = model.transform(test)

In [34]:
predictions.show()

+--------------------+-------+--------------------+--------------------+----------+
|            features|Outcome|       rawPrediction|         probability|prediction|
+--------------------+-------+--------------------+--------------------+----------+
|(8,[0,1,6,7],[2.0...|    0.0|[1.58287447954081...|[0.95952480924498...|       0.0|
|(8,[0,1,6,7],[7.0...|    0.0|[1.03617099568510...|[0.88818576039526...|       0.0|
|(8,[0,1,6,7],[10....|    1.0|[1.15502442384971...|[0.90970586778123...|       0.0|
|(8,[1,5,6,7],[99....|    0.0|[1.58688498588926...|[0.95983517528354...|       0.0|
|(8,[1,5,6,7],[117...|    0.0|[-0.8983873313062...|[0.14224413680957...|       1.0|
|[0.0,57.0,60.0,0....|    0.0|[0.72326462698802...|[0.80946370801305...|       0.0|
|[0.0,74.0,52.0,10...|    0.0|[1.27814675246510...|[0.92799518182705...|       0.0|
|[0.0,78.0,88.0,29...|    0.0|[1.22504119817087...|[0.92056747609235...|       0.0|
|[0.0,91.0,68.0,32...|    0.0|[-0.2936440124722...|[0.35725734895996...|    

In [36]:
# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="Outcome", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)


In [37]:
print ("Accuracy",accuracy)



Accuracy 0.759825327510917


In [38]:
spark.stop()